### Notes
- probably not enough data to use word to vec

In [6]:
# Imports

# Custom imports
from utils import accessors
from datetime import datetime as dt
import string
from tqdm.auto import tqdm
import collections

#import pandas as pd

In [2]:
show_outputs = True

In [3]:
path = "dataset/"
add_file = "addresses.jsonl"
cit_file = "cities.jsonl"

add_df = accessors.load_data(path, add_file) 
cit_df = accessors.load_data(path, cit_file) 

if show_outputs: 
    display(add_df)
    display(cit_df)

,address,country
0,Mitterweg Angath,AT
1,"josef-franz-huter-straße 72 6020, innsbruck, a...",AT
2,"FLIRSCHBERG FLIRSCH, ÖSTERREICH",AT
3,Lochbödele Landeck Austria,AT
4,"Ainet 75 9951, Ainet",AT
...,...,...
595491,RUE DU JEU DE PAUME VILLEPREUX,FR
595492,"78955, CARRIÈRES-SOUS-POISSY",FR
595493,rue du cèdre - morainvilliers,FR
595494,"Boissets, France",FR


,city,country
0,Kappl,AT
1,Schmirn,AT
2,Zams,AT
3,Wattenberg,AT
4,Polling in Tirol,AT
...,...,...
77476,Saint-Mesmin,FR
77477,Vernoux-en-Vivarais,FR
77478,Sedze-Maubecq,FR
77479,Bulat-Pestivien,FR


### Preprocess data

In [4]:
# Set all addresses and cities to lowercase
add_df['address'] = add_df['address'].str.lower()
cit_df['city'] = cit_df['city'].str.lower()

# Remove punctuation 
add_df['address'] = add_df['address'].str.replace('[{}]'.format(string.punctuation), '', regex=True)
cit_df['city'] = cit_df['city'].str.replace('[{}]'.format(string.punctuation), '', regex=True)

### Quick data analytics

In [5]:
# Any nans in the dataset?
print(add_df.isna().sum())
print(cit_df.isna().sum())

# Number of occurences of each country 
countries = add_df.groupby('country').count()
countries.plot.barh()
countries.plot.barh(log = True)
print(len(countries))
# (TODO: stratify train/test split over this distribtion)

address    0
country    0
dtype: int64
city       0
country    0
dtype: int64


KeyboardInterrupt: 

# Simple solution 
Split address into tokens and brute force search in database 

In [ ]:
t1 = dt.now()
# Split the address into a list
add_df['address_split'] = add_df['address'].str.split()

#add_df = add_df.head(20)

dupe_cities = collections.defaultdict(list)

# Loop over each row and
for i, row in tqdm(add_df.iterrows(), total=add_df.shape[0]): 
    address = row['address_split']
    for token in address: 
        if token.isnumeric(): 
            pass
        elif token in cit_df['city'].values:
            try: 
                add_df.loc[i, 'country_out'] = cit_df.loc[cit_df['city'] == token, 'country'].item()
            except: 
                dupe_cities[token].append(cit_df.loc[cit_df['city'] == token, 'country'].values)
            
display(add_df)

t2 = dt.now()
print("Time for calculation: {}".format((t2-t1)))

  0%|          | 0/595496 [00:00<?, ?it/s]

In [ ]:
add_df['country'] == add_df['country_out']

### Shortcomings
- not all addresses are following the same format, e.g. some have comma's, some do not. So end up with useless characters in your lists. Note, have now removed punctuation in preprocessing
- not all cities are one single string 
- special characters might change (e.g. ß or ss) 
- some cities exist in multiple countries